# Article to Trivia

### Step 1:  Use the following GraphQL query to get the full text of an article.

https://content-api.gannettdigital.com/graphiql?siteCode=USAT&type=asset&query=%7B%20asset%28id%3A%20%221365379001%22%29%20%7B%20...assetFields%20%7D%20%7D%20fragment%20assetFields%20on%20Asset%20%7B%20storyHighlights%20...%20on%20text%20%7B%20fullText%20%7D%20%7D%20

```
{
  asset(id: "1365379001") {
    ...assetFields
  }
}

fragment assetFields on Asset {
  ... on text {
    fullText
  }
}
```

### Step 2: Use the copy the value of full text and use the following website to strip the HTML

https://www.striphtml.com/

### Step 3: Paste the result into the variable below and Run All

In [9]:
text = "\n\nWEST POINT, N.Y. A West Point cadet was killed and 20 cadets and two soldiers injured Thursday when a tactical vehicle overturned on a dirt road duringa summer training exercise, according to the U.S. Military Academy.\n\nArmyLt. Col. Christopher Ophardt, an academy spokesman, said theaccident occurred around6:45 a.m. near West Point when the Light Medium Tactical Vehicle rolled.\n\nOphardt could not provide the severity of the injuries or the name of cadet who died. The injured were taken to localhospitals.\n\nThe incident occurreda dirt road off Route 293 at Camp Natural Bridge, a cadet summer training site near the main campus of the military academy.\n\n\n\nNew York Gov. Andrew Cuomo issued a statement expressing his condolences for the cadet who died and those injured.He directedthe state Office of Emergency Management to provide any resources necessary to assist to assist West Points efforts.\n\nMy heart breaks for all those involved in the tragic training accident at West Point this morning, Cuomo said. These courageous cadets and soldiers represent the best of New York State and our country, and we owe them a debt of gratitude for their bravery in choosing to serve our country and protect our freedoms.\n\n\n\nCuomo said theincident is made all the more heart-wrenching as we commemorate the 75th anniversary of D-Day today, a day where we remember those who gave their lives for our country.\n\nWest Point is located on the Hudson River about 60 miles north of New York City. It is a four-yearfederal service academy originally established as a fort and is one offive service academies in the U.S.\n\nCandidates for admission must both apply directly to the academy and receive a nomination.\n\nTom Vanden Brook reports from Washington;Peter Kramer and Steve LIeberman reportfor theRockland/Westchester Journal News\n\nContributing:The Associated Press\n"

In [10]:
print(text)



WEST POINT, N.Y. A West Point cadet was killed and 20 cadets and two soldiers injured Thursday when a tactical vehicle overturned on a dirt road duringa summer training exercise, according to the U.S. Military Academy.

ArmyLt. Col. Christopher Ophardt, an academy spokesman, said theaccident occurred around6:45 a.m. near West Point when the Light Medium Tactical Vehicle rolled.

Ophardt could not provide the severity of the injuries or the name of cadet who died. The injured were taken to localhospitals.

The incident occurreda dirt road off Route 293 at Camp Natural Bridge, a cadet summer training site near the main campus of the military academy.



New York Gov. Andrew Cuomo issued a statement expressing his condolences for the cadet who died and those injured.He directedthe state Office of Emergency Management to provide any resources necessary to assist to assist West Points efforts.

My heart breaks for all those involved in the tragic training accident at West Point this morni

In [22]:
import spacy

nlp = spacy.load("en_core_web_sm")

doc = nlp(text)

labels = ['GPE', 'ORG', 'PERSON', 'QUANTITY', 'FAC']

for entity in doc.ents:
    if(entity.label_ in labels):
        print(entity.text, entity.label_)

N.Y. GPE
West Point GPE
the U.S. Military Academy ORG
Christopher Ophardt PERSON
West Point GPE
the Light Medium Tactical Vehicle FAC
Route 293 FAC
Camp Natural Bridge FAC
New York GPE
Andrew Cuomo PERSON
Office of Emergency Management ORG
West Points GPE
West Point GPE
Cuomo ORG
New York State GPE
Cuomo ORG
West Point GPE
about 60 miles QUANTITY
New York City GPE
the U.S.

Candidates ORG
Tom Vanden Brook PERSON
Kramer PERSON
Steve LIeberman PERSON
theRockland/Westchester Journal News

Contributing: ORG
The Associated Press
 ORG
